In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
cwd = os.getcwd()
print(cwd)

/Users/rosamund/Documents/GitHub/uk-electricity-map


In [3]:
df1 = pd.read_csv((cwd + "/uk-map.csv"))
df1.head(3)

,Site,Company,Fuel,Type,Capacity (MW),Latitude,Longitude,Low-Carbon,CHP,Year Open,yearStart,yearEnd
0,Aberdare District Energy,EDF Energy,Gas,NaN,10.0,51.697527,-3.416181,No,NaN,2002.0,2005.0,2014
1,Aberthaw B,RWE Npower Plc,Coal,NaN,1455.0,51.387006,-3.406215,No,NaN,1971.0,2004.0,2007
2,Aberthaw B,RWE Npower Plc,Coal,NaN,1586.0,51.387006,-3.406215,No,NaN,1971.0,2008.0,2017


In [4]:
df1.tail(3)

,Site,Company,Fuel,Type,Capacity (MW),Latitude,Longitude,Low-Carbon,CHP,Year Open,yearStart,yearEnd
6970,Vital Energi,NaN,Gas,Gas turbine,4.560,NaN,NaN,No,Yes,NaN,2008.0,2019
6971,Weetabix,NaN,Gas,Reciprocating engine,6.058,NaN,NaN,No,Yes,NaN,2008.0,2019
6972,Wepa Ltd,NaN,Gas,Gas turbine,9.000,NaN,NaN,No,Yes,NaN,2008.0,2019


In [5]:
#think appearance of excess rows may due to Simon's treament of the data carrying across
#update: it's an issue with how Simon exported the CSV at row 1770

df1.shape

(6973, 12)

In [6]:
df1.tail(3)

,Site,Company,Fuel,Type,Capacity (MW),Latitude,Longitude,Low-Carbon,CHP,Year Open,yearStart,yearEnd
6970,Vital Energi,NaN,Gas,Gas turbine,4.560,NaN,NaN,No,Yes,NaN,2008.0,2019
6971,Weetabix,NaN,Gas,Reciprocating engine,6.058,NaN,NaN,No,Yes,NaN,2008.0,2019
6972,Wepa Ltd,NaN,Gas,Gas turbine,9.000,NaN,NaN,No,Yes,NaN,2008.0,2019


In [7]:
df2 = df1.rename(index=str, columns={"Site": "site", "Company": "operator", "Fuel": "type", "Type": "fuelDetail", "Capacity (MW)": "capacity", "Latitude": "latitude", "Longitude": "longitude", "Low-Carbon": "lowCarbon", "CHP": "chp", "Year Open": "yearOpen", "yearStart": "yearStart", "yearEnd": "yearEnd"})
df2.head(3)

,site,operator,type,fuelDetail,capacity,latitude,longitude,lowCarbon,chp,yearOpen,yearStart,yearEnd
0,Aberdare District Energy,EDF Energy,Gas,NaN,10.0,51.697527,-3.416181,No,NaN,2002.0,2005.0,2014
1,Aberthaw B,RWE Npower Plc,Coal,NaN,1455.0,51.387006,-3.406215,No,NaN,1971.0,2004.0,2007
2,Aberthaw B,RWE Npower Plc,Coal,NaN,1586.0,51.387006,-3.406215,No,NaN,1971.0,2008.0,2017


In [8]:
df2.tail(3)

,site,operator,type,fuelDetail,capacity,latitude,longitude,lowCarbon,chp,yearOpen,yearStart,yearEnd
6970,Vital Energi,NaN,Gas,Gas turbine,4.560,NaN,NaN,No,Yes,NaN,2008.0,2019
6971,Weetabix,NaN,Gas,Reciprocating engine,6.058,NaN,NaN,No,Yes,NaN,2008.0,2019
6972,Wepa Ltd,NaN,Gas,Gas turbine,9.000,NaN,NaN,No,Yes,NaN,2008.0,2019


In [9]:
#find out if any nulls for long/lat
sum(df2['longitude'].isna())

192

In [10]:
#drop rows where longitude is blank

df2 = df2.dropna(subset=['longitude'])
sum(df2['longitude'].isna())

0

In [11]:
#check latitude doesn't have NaN either

sum(df2['latitude'].isna())

0

In [12]:
#find out if any nulls for capacity
sum(df2['capacity'].isna())

1

In [13]:
#drop rows where capacity is blank

df2 = df2.dropna(subset=['capacity'])
sum(df2['capacity'].isna())

0

In [14]:
#shape as expected
df2.shape

(6780, 12)

In [15]:
#sort by capacity so smallest drawn last

df3 = df2.sort_values(by=['capacity'], ascending=False)

df3.head(3)

,site,operator,type,fuelDetail,capacity,latitude,longitude,lowCarbon,chp,yearOpen,yearStart,yearEnd
85,Drax,Drax Power Ltd,Coal,NaN,3870.0,53.737546,-0.999021,No,NaN,1974.0,2004.0,2012
91,Drax coal,Drax Power Ltd,Coal,NaN,3225.0,53.737546,-0.999021,No,NaN,1974.0,2013.0,2013
402,Hinkley Point C,EDF Energy,Nuclear,NaN,3200.0,51.206000,-3.144000,Yes,NaN,2026.0,2019.0,2019


In [16]:
#replace gaps with hyphens

df3 = df3.fillna("-")
df3.head(3)

,site,operator,type,fuelDetail,capacity,latitude,longitude,lowCarbon,chp,yearOpen,yearStart,yearEnd
85,Drax,Drax Power Ltd,Coal,-,3870.0,53.737546,-0.999021,No,-,1974,2004.0,2012
91,Drax coal,Drax Power Ltd,Coal,-,3225.0,53.737546,-0.999021,No,-,1974,2013.0,2013
402,Hinkley Point C,EDF Energy,Nuclear,-,3200.0,51.206000,-3.144000,Yes,-,2026,2019.0,2019


In [17]:
df3.tail(3)

,site,operator,type,fuelDetail,capacity,latitude,longitude,lowCarbon,chp,yearOpen,yearStart,yearEnd
3019,JB Wheaton,E.On,Storage,Battery,0.005,50.840997,-2.935433,Yes,-,2016,2016.0,2019
2998,Flat Holm Microgrid Project,Wind & Sun,Storage,Battery,0.005,51.378056,-3.122498,Yes,-,2006,2006.0,2019
196,Killingholme,Centrica,Gas,Combined cycle,0.000,53.653650,-0.255910,No,-,1994,2016.0,2016


In [18]:
#begin process of converting to geojson

import geopandas as gpd

In [19]:
from geopandas import GeoDataFrame
from shapely.geometry import Point

In [20]:
geometry = [Point(xy) for xy in zip(df3.longitude, df3.latitude)]
df3 = df3.drop(['longitude', 'latitude'], axis=1)
crs = {'init': 'epsg:4326'}
gdf = GeoDataFrame(df3, crs=crs, geometry=geometry)
gdf.head(3)

,site,operator,type,fuelDetail,capacity,lowCarbon,chp,yearOpen,yearStart,yearEnd,geometry
85,Drax,Drax Power Ltd,Coal,-,3870.0,No,-,1974,2004.0,2012,POINT (-0.9990209999999999 53.73754599999999)
91,Drax coal,Drax Power Ltd,Coal,-,3225.0,No,-,1974,2013.0,2013,POINT (-0.9990209999999999 53.73754599999999)
402,Hinkley Point C,EDF Energy,Nuclear,-,3200.0,Yes,-,2026,2019.0,2019,POINT (-3.144 51.206)


In [21]:
# NB need to delete old file before or throws error

gdf.to_file("data/power_stations.json", driver="GeoJSON")